# NutriAI Health Coach
Recomendaciones nutricionales personalizadas basadas en datos clínicos y una ontología sencilla.

In [3]:
from rdflib import Graph, Namespace, RDF
import pandas as pd

# Crear grafo y namespaces
g = Graph()
FOOD = Namespace("http://example.org/food/")
NUT = Namespace("http://example.org/nutrient/")
COND = Namespace("http://example.org/condition/")
EX = Namespace("http://example.org/ex/")

g.bind("food", FOOD)
g.bind("nut", NUT)
g.bind("cond", COND)
g.bind("ex", EX)

# Añadir alimentos y nutrientes
g.add((FOOD.Tofu, RDF.type, FOOD.PlantBasedProtein))
g.add((FOOD.Tofu, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Tofu, NUT.hasNutrient, NUT.Protein))

g.add((FOOD.Broccoli, RDF.type, FOOD.Vegetable))
g.add((FOOD.Broccoli, NUT.hasNutrient, NUT.Fiber))
g.add((FOOD.Broccoli, NUT.hasNutrient, NUT.Calcium))

g.add((FOOD.Sardines, RDF.type, FOOD.Fish))
g.add((FOOD.Sardines, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Sardines, NUT.hasNutrient, NUT.Protein))

g.add((FOOD.Lentils, RDF.type, FOOD.Legume))
g.add((FOOD.Lentils, NUT.hasNutrient, NUT.Fiber))
g.add((FOOD.Lentils, NUT.hasNutrient, NUT.Protein))

g.add((FOOD.Oats, RDF.type, FOOD.Grain))
g.add((FOOD.Oats, NUT.hasNutrient, NUT.Fiber))

g.add((FOOD.Spinach, RDF.type, FOOD.Vegetable))
g.add((FOOD.Spinach, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Spinach, NUT.hasNutrient, NUT.Iron))

g.add((FOOD.Milk, RDF.type, FOOD.Dairy))
g.add((FOOD.Milk, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Milk, NUT.hasNutrient, NUT.VitaminD))

# Beneficios
beneficios = [
    (NUT.Calcium, COND.Hypertension),
    (NUT.Calcium, COND.Osteoporosis),
    (NUT.Fiber, COND.Diabetes),
    (NUT.Protein, COND.MuscleLoss),
    (NUT.Iron, COND.Anemia),
    (NUT.VitaminD, COND.Osteoporosis)
]
for nutriente, condicion in beneficios:
    g.add((nutriente, EX.beneficialFor, condicion))

In [4]:
# Función de inferencia
def inferir_alimentos_para_condicion(condicion_uri):
    nutrientes_beneficiosos = set()
    for s, p, o in g.triples((None, EX.beneficialFor, condicion_uri)):
        nutrientes_beneficiosos.add(s)

    alimentos = []
    for alimento, _, nutriente in g.triples((None, NUT.hasNutrient, None)):
        if nutriente in nutrientes_beneficiosos:
            alimentos.append(str(alimento).split("/")[-1])
    return sorted(set(alimentos))

In [6]:
# Simulación de paciente y resultados
df_hipertension = pd.DataFrame(inferir_alimentos_para_condicion(COND.Hypertension), columns=["Alimentos recomendados para hipertensión"])
df_hipertension

,Alimentos recomendados para hipertensión
0,Broccoli
1,Milk
2,Sardines
3,Spinach
4,Tofu


In [7]:
df_hipertension.info

<bound method DataFrame.info of   Alimentos recomendados para hipertensión
0                                 Broccoli
1                                     Milk
2                                 Sardines
3                                  Spinach
4                                     Tofu>